In [1]:
import unittest
from unittest.mock import MagicMock
import numpy as np 
import importlib
import sys
sys.path.append('../')
from src import T_2_matrix, dissipation_tensor, pauli_basis_class
importlib.reload(T_2_matrix)
importlib.reload(dissipation_tensor)
importlib.reload(pauli_basis_class)
from src.T_2_matrix import T_2_Matrix
from src.dissipation_tensor import Dissipation_Tensor
from src.pauli_basis_class import Pauli_Basis
from scipy.sparse import lil_matrix

In [2]:
class T_2_Tensor_tests(unittest.TestCase):


    def test_constructor_argument_must_be_instance_of_dissipation_tensor(self):
        
        not_d_tensor = "This is not a dissipation tensor"
        
        with self.assertRaises(TypeError) as context:
            T_2_Matrix(not_d_tensor)
        
        self.assertEqual(str(context.exception), "d_tensor must be an instance of Dissipation_Tensor")



    def test_has_same_type_as_D_tensor_kossakowski(self):
        n = 15
        mock_structure_constants =  [lil_matrix((n,n), dtype=complex) for _ in range(n)]

        d_tensor_general = Dissipation_Tensor("general", mock_structure_constants, mock_structure_constants)
        d_tensor_symm = Dissipation_Tensor("symmetric", mock_structure_constants, mock_structure_constants)

        T_2_general = T_2_Matrix(d_tensor_general)
        T_2_symm = T_2_Matrix(d_tensor_symm)

        self.assertEqual(T_2_general.type, "general")
        self.assertEqual(T_2_symm.type, "symmetric")

    def test_if_correct_shape_depending_on_type(self):
        n = 15
        dof_symm = n + (n**2 - n)/2
        mock_structure_constants =  [lil_matrix((n,n), dtype=complex) for _ in range(n)]

        d_tensor_general = Dissipation_Tensor("general", mock_structure_constants, mock_structure_constants)
        d_tensor_symm = Dissipation_Tensor("symmetric", mock_structure_constants, mock_structure_constants)

        T_2_general = T_2_Matrix(d_tensor_general)
        T_2_symm = T_2_Matrix(d_tensor_symm)

        T_2_general_shape = T_2_general.shape
        T_2_symm_shape = T_2_symm.shape

        self.assertEqual(T_2_general_shape, (n**2, n**2))
        self.assertEqual(T_2_symm_shape, (n**2, dof_symm))

 

    def test_correctly_builds_general_t_2_tensor_for_su_2(self):
        
        n = 2
        my_pauli_basis = Pauli_Basis(n)
        complex_structure_constants = my_pauli_basis.complex_structure_constants()
        antisymmetric_structure_constants = my_pauli_basis.antisymmetric_structure_constants()

        my_dissipation_tensor = Dissipation_Tensor("general", complex_structure_constants, antisymmetric_structure_constants)

        my_t_2_matrix = T_2_Matrix(my_dissipation_tensor)

        output = my_t_2_matrix.matrix.toarray()
        expected = np.array([[ 
                            0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, -1. -0.j,  0. +0.j,
                                0. +0.j,  0. +0.j, -1. -0.j],
                            [ 0. +0.j,  0.5-0.j,  0. +0.j,  0.5-0.j,  0. +0.j,  0. +0.j,
                                0. +0.j,  0. +0.j,  0. +0.j],
                            [ 0. +0.j,  0. +0.j,  0.5-0.j,  0. +0.j,  0. +0.j,  0. +0.j,
                                0.5-0.j,  0. +0.j,  0. +0.j],
                            [ 0. +0.j,  0.5-0.j,  0. +0.j,  0.5-0.j,  0. +0.j,  0. +0.j,
                                0. +0.j,  0. +0.j,  0. +0.j],
                            [-1. -0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
                                0. +0.j,  0. +0.j, -1. -0.j],
                            [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5-0.j,
                                0. +0.j,  0.5-0.j,  0. +0.j],
                            [ 0. +0.j,  0. +0.j,  0.5-0.j,  0. +0.j,  0. +0.j,  0. +0.j,
                                0.5-0.j,  0. +0.j,  0. +0.j],
                            [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5-0.j,
                                0. +0.j,  0.5-0.j,  0. +0.j],
                            [-1. -0.j,  0. +0.j,  0. +0.j,  0. +0.j, -1. -0.j,  0. +0.j,
                                0. +0.j,  0. +0.j,  0. +0.j]])
                        
        self.assertTrue(np.allclose(output, expected))

    def test_correctly_builds_symmetric_t_2_tensor_for_su_2(self):
        
        n = 2
        my_pauli_basis = Pauli_Basis(n)
        complex_structure_constants = my_pauli_basis.complex_structure_constants()
        antisymmetric_structure_constants = my_pauli_basis.antisymmetric_structure_constants()

        my_dissipation_tensor = Dissipation_Tensor("symmetric", complex_structure_constants, antisymmetric_structure_constants)

        my_t_2_matrix = T_2_Matrix(my_dissipation_tensor)

        output = my_t_2_matrix.matrix.toarray()

        expected = np.array([
                    [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j, -1.+0.j],
                    [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
                    [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
                    [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
                    [-1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j],
                    [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
                    [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
                    [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
                    [-1.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j]])


        self.assertTrue(np.allclose(output,expected.real))
    
    def test_compute_correct_rank(self):

        n = 4
        my_pauli_basis = Pauli_Basis(n)
        complex_structure_constants = my_pauli_basis.complex_structure_constants()
        antisymmetric_structure_constants = my_pauli_basis.antisymmetric_structure_constants()

        k_gen = "general"
        k_symm = "symmetric"
        my_dissipation_tensor_gen = Dissipation_Tensor(k_gen, complex_structure_constants, antisymmetric_structure_constants)
        my_dissipation_tensor_symm = Dissipation_Tensor(k_symm, complex_structure_constants, antisymmetric_structure_constants)

        my_t_2_matrix_gen = T_2_Matrix(my_dissipation_tensor_gen)
        my_t_2_matrix_symm = T_2_Matrix(my_dissipation_tensor_symm)

        gen_rank = my_t_2_matrix_gen.rank
        symm_rank = my_t_2_matrix_symm.rank

        self.assertEqual(210, gen_rank)
        self.assertEqual(120, symm_rank)


    def test_is_matrix_full_rank(self):
        n = 2
        my_pauli_basis = Pauli_Basis(n)
        complex_structure_constants = my_pauli_basis.complex_structure_constants()
        antisymmetric_structure_constants = my_pauli_basis.antisymmetric_structure_constants()
        my_dissipation_tensor = Dissipation_Tensor("symmetric", complex_structure_constants, antisymmetric_structure_constants)
        my_t_2_matrix = T_2_Matrix(my_dissipation_tensor)
        
        output = my_t_2_matrix.isfullrank()
        
        self.assertTrue(output)




In [3]:
unittest.main(argv=[''], exit = False)

C:\Users\waqas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_lil.py:258: ComplexWarning: Casting complex values to real discards the imaginary part
  x = self.dtype.type(x)
.......
----------------------------------------------------------------------
Ran 7 tests in 153.730s

OK
